# 節點數量

In [ ]:
import json
import pandas as pd
from collections import OrderedDict

# 讀取 JSON 檔案
with open('result.json', 'r', encoding='utf-8') as file:
    data = json.load(file)

count_dict = {}
for row in data:
    for link in row["links"]:
        # 檢查 link[1] 是否已經作為一個鍵存在於 count_dict 中
        if link[1] in count_dict:
            # 如果存在，更新 count
            count_dict[link[1]]["count"] += 1
        else:
            # 如果不存在，創建一個新的條目
            count_dict[link[1]] = {"count": 1, "title": row["title"]}   
sorted_count_dict = OrderedDict(sorted(count_dict.items(), key=lambda x: x[1]['count'], reverse=True))

print(f"原始節點數量: {len(data)}")
print(f"延伸節點數量: {len(sorted_count_dict)}")


# Embedding Model Evaluation

In [ ]:
from transformers import AutoTokenizer, AutoModel
import torch
import torch.nn.functional as F

# 載入模型和分詞器
model_names = [ "sentence-transformers/paraphrase-mpnet-base-v2",
                "sentence-transformers/multi-qa-mpnet-base-dot-v1", 
                "sentence-transformers/multi-qa-MiniLM-L6-cos-v1",
                "sentence-transformers/all-MiniLM-L12-v2",
                "sentence-transformers/all-MiniLM-L6-v2",
                "sentence-transformers/all-mpnet-base-v2",
                "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"]

for model_name in model_names:

    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModel.from_pretrained(model_name)
    def mean_pooling(model_output, attention_mask):
        token_embeddings = model_output[0]  # 取得所有token的嵌入向量
        input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
        return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)

    def get_sentence_embedding(sentence):
        # 分詞並準備模型輸入
        encoded_input = tokenizer(sentence, return_tensors='pt', truncation=True)

        # 計算嵌入向量
        with torch.no_grad():
            model_output = model(**encoded_input)

        # 取平均池化作為句子向量
        attention_mask = encoded_input['attention_mask']
        sentence_embedding = mean_pooling(model_output, attention_mask)

        # 正規化
        sentence_embedding = F.normalize(sentence_embedding, p=2, dim=1)

        return sentence_embedding.squeeze()

    def cosine_similarity(a, b):
        dot_product = torch.dot(a, b)
        
        norm_a = torch.norm(a, p=2)
        norm_b = torch.norm(b, p=2)
        
        similarity = dot_product / (norm_a * norm_b)
        return similarity

    # 獲得詞嵌入
    word_embedding1 = get_sentence_embedding('張立杰教授 簡歷')
    word_embedding2 = get_sentence_embedding('規章與辦法 – 中央大學|學習與教學研究所 【行政】 1. 2. 3. 【教師】   【學生】')
    word_embedding3 = get_sentence_embedding('張立杰教授 – 中央大學|學習與教學研究所 國立中央大學|學習與教學研究所 Graduate Institute of Learning and Instruction, National Central University, Taiwan Toggle Navigation 首頁 最新消息 關於學習所 本所簡介 規章與辦法 規章與辦法 教師表格下載 學生表格下載 行政與聯絡 師資與研究 教師介紹 研究主題與方法 學生與校友 學生 出國開會心得 校友研究 課程 修業辦法 課程地圖與開課紀錄 師培中心 網路資源 教室租借 網站連結 English 張立杰教授 Li-Chieh Chang, Professor (03) 4227151 ext 33870 bchang.tw@gmail.com 個人網站 Bio/簡歷 中央大學、師培中心主任 (2020.8 ~ ) 中央大學、學習與教學研究所所長 (2020.8 ~ ) 中央大學、師資培育中心/學習與教學研究所、教授 (2019.08 ~ ) 台灣永齡希望小學中大分校負責人 (2014 ~ ) 中央大學、教務處、註冊組組長 (2018.8 ~2021.1) 中央大學、師資培育中心、主任 (2014.08 ~ 2017.07) 中央大學、師資培育中心/學習與教學研究所、副教授 (2013.08 ~2019.07) 嘉義大學、數位學習設計與管理學系、副教授 (2011.08 ~2013.07 ) 嘉義大學、教學發展中心、教學科技與學習輔導組組長 (2009.02 ~ 2012.02) 嘉義大學、數位學習設計與管理學系、助理教授 (2008.02 ~ 2011.07) 中央大學、學習科技研究中心、博士後研究 (2006.07 ~ 2008.01) 美國史丹福大學、創新學習科技中心、訪問博士後研究 (2005.08 ~ 2006.07) 中央大學、學習科技研究中心、博士後研究 (2003.10 ~ 2005.07) Teaching/教學 109-2 自然領域探究與實作 科技與教學整合專題研究 109-1 書報討論 教育實習 108-2 自然領域探究與實作 通識中心 – 科技與閱讀寫作 108-1 科技與教學整合專題研究 教育實習 107-2 教育議題專題 教學媒體與操作 107-1 科技與教學創新整合專題研究 生活科技概論 教育實習 Research/研究 計畫: 2018-2021: 社會學習網絡之自動化知識關聯分析及建構 (MOST 107-2511-H-008 -006 -MY3) 2016-2018: 關聯式設計策略：概念知識雲及其社會學習網絡之建置 (MOST 105-2511-S-008 -007 -MY2) 2013-2016: 基於教室學習網絡設計之遊戲化 Apps 及其人因分析 (MOST 102-2511-S-008 -019 -MY3) 2012: 從認知風格探討科技學習工具介入之群組式數位學習環境之影響 (MOST 101-2511-S-415-016-) 2010 – 2011: 教室中多人共用一機之競爭學習情境設計及學生喜好性分析 (99-2511-S-415-006-MY2) 2009: 使用廣域無線感測網路支援氣象科學探究活動 (98-2511-S-415-008) 2008: 教室內無線感測網路支援群體手勢活動之互動學習環境設計 (97-2511-S-415-013) 論文: 28.Shih, YA., & Chang, B. (2020). Empirical study on the effects of social network–supported group concept mapping. RPTEL 15, 23 (2020). https://doi.org/10.1186/s41039-020-00143-7 27.Shih, Y., Chang, B., & Chin, J. Y. (2020). Data-driven student homophily pattern analysis of online discussion in a social network learning environment. Journal of Computers in Education, 7, 373-394. doi: https://doi.org/10.1007/s40692-020-00160-x 26.Chan, T. W., Looi, C. K., Chang, B., Chen, W., Wong, L. H., Wong, S. L., … Chou, C. Y. (2019). IDC theory: Creation and the creation loop. RPTEL 14, 26. doi:10.1186/s41039-019-0120-5 25.Chan, T. W., Looi, C. K., Chen, W., Wong, L. H., Chang, B., Liao, C. C. Y., … Ogata, H. (2018). Interest-driven creator theory: towards a theory of learning design for Asia in the twenty-first century. Journal of Computers in Education, 5(4), 435-461. 24.Chang, B., Shih, Y. A., & Lu, F. C. (2018). Co-construction concept through cloud-based social network platform design, implementation, and evaluation. The International Review of Research in Open and Distributed Learning, 19(5), 238-253. (SSCI) doi: http://dx.doi.org/10.19173/irrodl.v19i5.3419 23.Chang, B., & Lu, F. C. (2018). Social media facilitated English prewriting activity design and evaluation. The Asia-Pacific Education Researcher, 27(1), 33-42. (SSCI) doi: 10.1007/s40299-017-0363-0 22.Yu, F. Y., Hsieh, H. T., & Chang, B. (2017). The potential of Second Life for university counseling: A comparative approach examining media features and counseling problems. Research and Practice in Technology Enhanced Learning.12:24. doi: 10.1186/s41039-017-0064-6 21.Chang, B., Chen, S., Tsai, Y. C., & Lai, M. L. (2016). The effects of task activities and gaming scales on eye and visual search performance. Computers in Human Behavior, 66, 16-25. (SSCI) 20.Yang, E. F. Y., Chang, B., Cheng, H. N. H., & Chan, T. W. (2016). Improving pupils’ mathematical communication abilities through computer-supported reciprocal peer tutoring. Educational Technology & Society, 19(3), 157-169. (SSCI) 19.Lu, F. C., & Chang, B. (2016). Role-play game enhanced English for specific purpose vocabulary acquisition framework. Educational Technology & Society, 19(2), 367-377. (SSCI) 18.Wang, J. H., Chen, S. Y., Chang, B., & Chan, T. W. (2016). From integrative peer response to game-based integrative peer response: High ability vs. low ability. Journal of Computer Assisted Learning, 32(2), 170-185. (SSCI) 17.Cheng, H. N. H., Yang, E. F. Y., Liao, C. C. Y., Chang, B., Huang, Y. C. Y., & Chan, T. W. (2015). Scaffold seeking: A reverse design of scaffolding in computer-supported word problem solving. Journal of Educational Computing Research, 53, 409-435. (SSCI) 16.Chang, B., Chen, S., & Jhan, S.N. (2015). The influences of an interactive group-based videogame: Cognitive styles vs. prior ability. Computers & Education, 88, 399-407. (SSCI) 15.Chuang, M. T., Chang, B., & Chen, H. C. (2014). Investigating reported social and affective strategy used by EFL learners in virtual and real learning environments. International Journal of Applied Linguistics & English Literature, 3(2), 27-33. 14.Chang, B., Yu, F. Y., Chen, Y. Y., & Hsieh, H. T. (2013). Thinking in an opposing position: A framework design of role-reversal pedagogy through technology. Research and Practice in Technology Enhanced Learning, 8(3), 347-362. 13.Chang, B., Chuang, M. T., & Ho, S. (2013). Understanding students’ competition preference in multiple-mice supported classroom. Educational Technology & Society, 16(1), 171-182. (SSCI) 12.Yu, F. Y., Hsieh, H. T., & Chang B. (2011). Potential of second life for psychological counseling: A comparative approach. Lecture Notes in Computer Science, 6872, 44–45. (EI) 11.Chang, B., Wang, H. Y., Peng, T. Y., & Hsu, Y. S. (2010). Development and evaluation of a city-wide wireless weather sensor network. Educational Technology & Society, 13(3), 270-280. (SSCI) 10.Elwell, M., & Chang, B. (2010, January). Mission-oriented situated second language learning in Second Life. IEEE Learning Technology Newsletter, 12(1), 62-65. 9.Chang, S. B., Lin, C. J., Ching, E., Cheng, H. N. H., Chang, B., Chen, F. C., … Chan, T. W. (2009). EduBingo: Developing a content sample for the one-to-one classroom by the content-first design approach. Educational Technology & Society, 12(3), 343-353. (SSCI) 8.Wang, H. Y., Chang, B. & Harn, C. T. (2009, July). Using mobile data logger and city-wide weather database to facilitate parent-child coupled climatology learning. IEEE Learning Technology Newsletter, 11(3), 4-6. 7.Chang, B., Wang, H. Y. & Lin, Y. S. (2009, January-April). Enhancement of mobile learning using wireless senor network. IEEE Learning Technology Newsletter, 11(1&2), 22-25. 6.Chang, B., Cheng, N. H., Deng, Y. C. & Chan, T. W. (2007). Environmental design for a structured network learning society. Computers and Education, 48(2), 234-249. (SSCI, EI) 5.Liang, J. K., Liu, T. C., Wang, H. Y., Chang, B., Deng, Y. C., Yang, J. C., Chou, C. Y., Ko, H. W., Yang, S., & Chan, T. W. (2005). A few design perspectives on one-on-one digital classroom. Journal of Computer-Assisted Learning, 21(3), 181-189. (SSCI) 4.Chang, L. J., Chou, C. Y., Chen, Z. H., & Chan, T. W. (2004). An approach to assisting teachers in building physical and network hybrid community-based learning environments: The Taiwanese experience. Journal of Educational Development, 24(4), 361-381. (SSCI) 3.Chang, L. J., Yang, J. C., Deng, Y. C., & Chan, T. W. (2003). EduXs: Multilayer educational services platforms. Computers & Education, 41(1), 1-18. (SSCI, EI) 2.Chang, L. J., Yang, J. C., Yu, F. Y., & Chan, T. W. (2003). Development and evaluation of multiple competitive activities in a synchronous quiz game system. Journal of Innovations in Education and Training International, 40(1), 16-26. (SSCI) 1.Yu, F. Y., Chang, L. J., Liu, Y. H., & Chan, T. W. (2002). Learning preferences and satisfaction of senior high school students toward different sunchronized computerized competitive modes. Journal of Computer-Assisted Learning, 18(3), 341-350. (SSCI)   桃園巿中壢區中大路300號 人文社會科學大樓LS-401室 中央大學學習與教學研究所 Phone: 886-3-4227151 # 33851 Fax: 886-3-4273371 Email: ncu3851@ncu.edu.tw 中央大學入口網站 MyNCU © 2017 學習與教學研究所. All Rights Reserved. About this site Kale, A Food Blog Theme by LyraThemes.com.')

    # 計算相似度
    similarity_1_2 = cosine_similarity(word_embedding1, word_embedding2)
    similarity_1_3 = cosine_similarity(word_embedding1, word_embedding3)
    similarity_2_3 = cosine_similarity(word_embedding2, word_embedding3)

    print(model_name, " => dim:", len(word_embedding1))
    print(f"Similarity between word_1_2:", similarity_1_2.item())
    print(f"Similarity between word_1_3:", similarity_1_3.item())
    print(f"Similarity between word_2_3", similarity_2_3.item())

    print("--------------------------")
    
# 預設 1_3 應該要最好